# Capitulo 1 - Introdução

Nesse capítulo serão apresentados conceitualmente os conceito centrais do trabalhos do cientista de dados a partir de uma base de dados gerada manualmente. Essa base se baseia em uma empresa fictícia que gerencia uma rede social voltada para cientistas de dados.

In [1]:
from collections import Counter, defaultdict
import seaborn as sns
import pandas as pd

## Contatos e amizades

Esse bloco trata essencialmente de uma lista de usuários e suas conexões. Os processos aqui efetuados buscam explorar a lista de contatos, entendendo as relações básicas entre os usuários. Algumas métricas calculadas são o tamanho da base, as relações de cada indivíduo da base com o restante e o número médio de conexões efetuadas, além de fazer uma primeira aproximação em um algoritmo de recomendação de usuários baseados no conceito de "friend of a friend" (ou foaf).

In [2]:
usuarios = [
    {"id" : 0, "name" : "Hero"},
    {"id" : 1, "name" : "Dunn"},
    {"id" : 2, "name" : "Sue"},
    {"id" : 3, "name" : "Chi"},
    {"id" : 4, "name" : "Thor"},
    {"id" : 5, "name" : "Clive"},
    {"id" : 6, "name" : "Hicks"},
    {"id" : 7, "name" : "Devin"},
    {"id" : 8, "name" : "Kate"},
    {"id" : 9, "name" : "Klein"},
]

pares_de_conexoes = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [3]:
amizades = {usuario['id']: [] for usuario in usuarios}
amizades                                                                                                                

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

In [4]:
# Lista todas as conexoes de cada usuário

for i, j in pares_de_conexoes:
    amizades[i].append(j)
    amizades[j].append(i)

amizades

{0: [1, 2],
 1: [0, 2, 3],
 2: [0, 1, 3],
 3: [1, 2, 4],
 4: [3, 5],
 5: [4, 6, 7],
 6: [5, 8],
 7: [5, 8],
 8: [6, 7, 9],
 9: [8]}

In [5]:
# Conta quantas conexões existem na rede

def numero_de_amigos(usuario):
    usuario_id = usuario['id']
    friends_ids = amizades[usuario_id]
    return len(friends_ids)

total_de_conexoes = sum(numero_de_amigos(usuario)
                        for usuario in usuarios)

# Calcula média de conexões por usuário

num_de_usuarios = len(usuarios)
media_de_amizades = total_de_conexoes / num_de_usuarios
media_de_amizades

2.4

In [6]:
# Encontra o usuário com mais conexões

numero_de_amigos_por_id = [(usuario["id"], numero_de_amigos(usuario))
                            for usuario in usuarios]

numero_de_amigos_por_id.sort(key = lambda id_e_amigos: id_e_amigos[1], reverse = True)
numero_de_amigos_por_id

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

In [7]:
# Usuários que talvez você conheça

def foaf_ids(usuario):
    return [foaf_id 
            for id_conexao in amizades[usuario["id"]]
            for foaf_id in amizades[id_conexao]]

print(amizades[1])
print(amizades[2])

print(foaf_ids(usuarios[0]))

[0, 2, 3]
[0, 1, 3]
[0, 2, 3, 0, 1, 3]


In [8]:
def amigo_do_amigo(usuario):
    usuario_id = usuario["id"]
    return Counter(
        foaf_id
        for id_conexao in amizades[usuario_id]
        for foaf_id in amizades[id_conexao]
        if foaf_id != usuario_id and foaf_id not in amizades[usuario_id]
    )

print(amigo_do_amigo(usuarios[3]))

Counter({0: 2, 5: 1})


## Interesses

Esse segundo bloco trata do conceito de assuntos de interesse de cada usuário. No começo temos uma tabela que lista quais os assuntos preferidos de cada usuário. Depois, temos uma série de buscas por padrões e entendimento da base, com conexões entre os usuários, busca de nichos mais interessantes dentre outros.

In [9]:
# Cria uma separação por interesses para uma recomendação mais precisa

interesses = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodel"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial inteligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

In [10]:
def cientistas_de_dados_que_gostam(assunto):
    return[id_usuario
           for id_usuario, interesse_do_usuario in interesses
           if interesse_do_usuario == assunto]

users_ids_por_interesse = defaultdict(list)

for id_usuario, interesse in interesses:
    users_ids_por_interesse[interesse].append(id_usuario)

users_ids_por_interesse

defaultdict(list,
            {'Hadoop': [0, 9],
             'Big Data': [0, 8, 9],
             'HBase': [0, 1],
             'Java': [0, 5, 9],
             'Spark': [0],
             'Storm': [0],
             'Cassandra': [0, 1],
             'NoSQL': [1],
             'MongoDB': [1],
             'Postgres': [1],
             'Python': [2, 3, 5],
             'scikit-learn': [2, 7],
             'scipy': [2],
             'numpy': [2],
             'statsmodel': [2],
             'pandas': [2],
             'R': [3, 5],
             'statistics': [3, 6],
             'regression': [3, 4],
             'probability': [3, 6],
             'machine learning': [4, 7],
             'decision trees': [4],
             'libsvm': [4],
             'C++': [5],
             'Haskell': [5],
             'programming languages': [5],
             'mathematics': [6],
             'theory': [6],
             'Mahout': [7],
             'neural networks': [7, 8],
             'deep learning': [

In [11]:
interesses_por_id_usuario = defaultdict(list)

for id_usuario, interesse in interesses:
    interesses_por_id_usuario[id_usuario].append(interesse)

interesses_por_id_usuario

defaultdict(list,
            {0: ['Hadoop',
              'Big Data',
              'HBase',
              'Java',
              'Spark',
              'Storm',
              'Cassandra'],
             1: ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres'],
             2: ['Python',
              'scikit-learn',
              'scipy',
              'numpy',
              'statsmodel',
              'pandas'],
             3: ['R', 'Python', 'statistics', 'regression', 'probability'],
             4: ['machine learning', 'regression', 'decision trees', 'libsvm'],
             5: ['Python',
              'R',
              'Java',
              'C++',
              'Haskell',
              'programming languages'],
             6: ['statistics', 'probability', 'mathematics', 'theory'],
             7: ['machine learning',
              'scikit-learn',
              'Mahout',
              'neural networks'],
             8: ['neural networks',
              'deep learning',
        

In [12]:
def assuntos_mais_comuns(user):
    return Counter(
        interesse_por_id_usuario
        for interesse in interesses_por_id_usuario[user['id']]
        for interesse_por_id_usuario in users_ids_por_interesse[interesse]
        if interesse_por_id_usuario != user['id']
    )

assuntos_mais_comuns(usuarios[0])

Counter({9: 3, 1: 2, 8: 1, 5: 1})

In [13]:
contagem_de_palavras = Counter(palavra 
                               for user, interesse in interesses
                               for palavra in interesse.lower().split())

contagem_de_palavras

Counter({'big': 3,
         'data': 3,
         'java': 3,
         'python': 3,
         'learning': 3,
         'hadoop': 2,
         'hbase': 2,
         'cassandra': 2,
         'scikit-learn': 2,
         'r': 2,
         'statistics': 2,
         'regression': 2,
         'probability': 2,
         'machine': 2,
         'neural': 2,
         'networks': 2,
         'spark': 1,
         'storm': 1,
         'nosql': 1,
         'mongodb': 1,
         'postgres': 1,
         'scipy': 1,
         'numpy': 1,
         'statsmodel': 1,
         'pandas': 1,
         'decision': 1,
         'trees': 1,
         'libsvm': 1,
         'c++': 1,
         'haskell': 1,
         'programming': 1,
         'languages': 1,
         'mathematics': 1,
         'theory': 1,
         'mahout': 1,
         'deep': 1,
         'artificial': 1,
         'inteligence': 1,
         'mapreduce': 1})

## Salários

Este pequeno bloco busca algumas relações entre experiência do usuário e seu salário médio, apontando uma correlação positiva esperada entre as variáveis (usuários com mais experiência tendem a ganhar mais).

In [14]:
salario_e_experiencia = [
    (83000, 8.7), (88000, 8.1),
    (48000, 0.7), (76000, 6),
    (69000, 6.5), (76000, 7.5),
    (60000, 2.5), (83000, 10),
    (48000, 1.9), (63000, 4.2)]

salario_por_experiencia = defaultdict(list)

for salario, experiencia in salario_e_experiencia:
    salario_por_experiencia[experiencia].append(salario)

salario_por_experiencia

defaultdict(list,
            {8.7: [83000],
             8.1: [88000],
             0.7: [48000],
             6: [76000],
             6.5: [69000],
             7.5: [76000],
             2.5: [60000],
             10: [83000],
             1.9: [48000],
             4.2: [63000]})

In [15]:
media_salario_por_experiencia = {experiencia : sum(salarios) / len(salarios)
                                 for experiencia, salarios in salario_por_experiencia.items()}

media_salario_por_experiencia

{8.7: 83000.0,
 8.1: 88000.0,
 0.7: 48000.0,
 6: 76000.0,
 6.5: 69000.0,
 7.5: 76000.0,
 2.5: 60000.0,
 10: 83000.0,
 1.9: 48000.0,
 4.2: 63000.0}

In [16]:
def agrupa_salarios(experiencia):
    if experiencia < 2:
        return "Júnior"
    elif experiencia < 5:
        return "Pleno"
    else:
        return "Sênior"
    
salario_por_experiencia_agrupada = defaultdict(list)

for salario, experiencia in salario_e_experiencia:
    grupo = agrupa_salarios(experiencia)
    salario_por_experiencia_agrupada[grupo].append(salario)

salario_medio_agrupado = {
    experiencia_agrupada : sum(salario) / len(salario)
    for experiencia_agrupada, salario in salario_por_experiencia_agrupada.items()
}

salario_medio_agrupado

{'Sênior': 79166.66666666667, 'Júnior': 48000.0, 'Pleno': 61500.0}

## Contas pagas

Esse último bloco cria uma correlação semi-arbitrária sobre a relação entre as variáveis "tempo de experiência" e "contas pagas". Essa relação não foi calculada, mas intuida a partir da observação direta dos dados. Ou seja, vê-se um padrão nas informações que será universalizado. Naturalmente essa abordagem não é a ideal, nem sequer é positiva, mas é uma abordagem preliminar sobre um problema que será estudado com mais detalhes futuramente.

In [17]:
contas_pagas = {
    0.7 : "pago",
    1.9 : "não pago",
    2.5 : "pago",
    4.2 : "não pago",
    6.0 : "não pago",
    6.5 : "não pago",
    7.5 : "não pago",
    8.1 : "não pago",
    8.7 : "pago",
    10.0 : "pago"
}

In [18]:
def preve_pagamento(tempo_de_experiencia):
    if tempo_de_experiencia < 3:
        return "pago"
    elif tempo_de_experiencia < 8.5:
        return "não pago"
    else:
        return "pago"